Project Minor A.I.
Hogeschool Rotterdam
by Rogier Mangoensentono


# Movie Review Sentiment Analysis via LSTM + Clustering

## Introductie

**@Tony Busker: Zie hier de bijbehorende Word document indien u deze niet in mijn e-mail hebt gezien: https://rogierr2.stackstorage.com/s/rngyQ5ADf3TOtGD**

### Beschrijving
De opdracht is om een A.I. te maken die de aard van een film beoordeling ziet. Dit doe ik d.m.v. de algoritmes **LSTM (Long Short-Term Memory)** en **Clustering**.

Hieronder staan de vijf verschillende sentimenten:

* 0 - negative
* 1 - somewhat negative
* 2 - neutral
* 3 - somewhat positive
* 4 - positive

Dit is mijn allereerste keer dat ik zo'n data analyse ga doen. Via Kaggle kan ik geüploade datasets gebruiken en meteen online ermee aan de slag gaan, wat het werk een stuk toegankelijker en makkelijker maakt. 

# Plan van aanpak
### Import de data sets
Import de gegeven data sets

### Visualiseer de data
Door het visualiseren van de data kan ik een beeld vormen ervan. Ik heb dan alvast een idee en ik kan een verwachting hebben van het resultaat.

### Cluster de data 
Via deze clusters kan ik een model maken waarop ik LSTM op kan toepassen.
http://colah.github.io/posts/2015-08-Understanding-LSTMs/
https://towardsdatascience.com/illustrated-guide-to-lstms-and-gru-s-a-step-by-step-explanation-44e9eb85bf21

LSTM maakt het mogelijk om een voorspelling te doen op een grote stuk tekst, zoals een beoordeling van een film. Hiervoor slaat het tijdelijk relevante data op en "vergeet" het minder belangrijke data.

### Het maken van het model voor LSTM met Clustering
Volgens https://towardsdatascience.com/how-to-build-a-data-set-for-your-machine-learning-project-5b3b871881ac moet je de train set verdelen tot een train set en een validation set.
Hier moet ik bepalen welke data relevant is en wat onthouden moet worden door de computer.

### Output
Uiteindelijk moet de prediction in de sampleSubmission file komen te staan, die gegeven is door Kaggle.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np
import numpy # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import string
from textblob import TextBlob
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize

from nltk import FreqDist
from nltk.stem import SnowballStemmer,WordNetLemmatizer
stemmer=SnowballStemmer('english')
lemma=WordNetLemmatizer()
from string import punctuation
import re


import gc
from keras.preprocessing import sequence,text
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense,Dropout,Embedding,LSTM,Conv1D,GlobalMaxPooling1D,Flatten,MaxPooling1D,GRU,SpatialDropout1D,Bidirectional
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report,f1_score

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

### Read de Train en Test Sets

In [ ]:
train = pd.read_csv("../input/train.tsv",sep="\t")
test = pd.read_csv("../input/test.tsv",sep="\t")

In [ ]:
sub = pd.read_csv('../input/sampleSubmission.csv', sep=",")

In sub komen de uiteindelijke resultaten. Deze komen in de vorm van (phraseId, sentiment).


In [ ]:
train.head()

De A.I. kan dus al de verschillende sentimenten zien.

In [ ]:
test.head()

 ### Clean de zinnen d.m.v. leestekens en trefwoorden (lemma)

In [ ]:
def clean_review(reviews):
    reviews_clean = []
    for i in range(0, len(reviews)):
        review = str(reviews[i])
        review = re.sub('[^a-zA-Z]', ' ', review) # regular expression
        review = [
            lemma.lemmatize(w) 
            for w in word_tokenize(str(review).lower())
        ]
        review=' '.join(review)
        reviews_clean.append(review)
        
    return reviews_clean

In [ ]:
train['CleanedPhrase'] = clean_review(train.Phrase.values)
train.head()

In [ ]:
test['CleanedPhrase'] = clean_review(test.Phrase.values)
test.head()

### Tel de woorden in CleanedPhrase d.m.v. TextBlob library


In [ ]:
train['WordCount'] = train['CleanedPhrase'].apply(lambda x: len(TextBlob(x).words))
test['WordCount'] = test['CleanedPhrase'].apply(lambda x: len(TextBlob(x).words))

In [ ]:
train = train[train['WordCount'] >= 1]
train = train.reset_index(drop = True)

In [ ]:
train.head()

In [ ]:
test.head()

### Een overzicht van de hoeveelheid woorden per sentiment

In [ ]:
sentiment_overview = train.groupby('Sentiment')['WordCount'].describe().reset_index()
sentiment_overview

Er zijn veel meer woorden met een "neutrale (2)" sentiment: bijna 80.000. Terwijl de één na hoogste aantal woorden zijn bij een "lichtelijk positieve (3)" sentiment, bijna 33.000. Het verschil tussen deze twee is dus best groot.

### Aantal per sentiment grafiek
Het is misschien ook interessant om deze data in een grafiek te doen. 

Dit kunnen we doen door gebruik te maken van de libraries plotly en matplot.

In [ ]:
import plotly.offline as py
py.init_notebook_mode(connected=True)
from plotly.offline import init_notebook_mode, iplot
import plotly.figure_factory as ff
import matplotlib as plt
import plotly.graph_objs as go
import plotly.tools as tls
%matplotlib inline
import cufflinks as cf
cf.go_offline()

### Bar graph of min, mean and max sentence lenght of each sentiment wise
Bar graph represent all sentence length are eqully distribute for each sentiment wise

In [ ]:
min_length = go.Bar(
    x = sentiment_overview['Sentiment'],
    y = sentiment_overview['min'],
    name = 'Min zinslength'
)

average_length = go.Bar(
    x = sentiment_overview['Sentiment'],
    y = sentiment_overview['mean'],
    name = 'Average Sentence length'
)

max_length = go.Bar(
    x = sentiment_overview['Sentiment'],
    y = sentiment_overview['max'],
    name = 'Max Sentence length'
)

data = [min_length, average_length, max_length]
layout = go.Layout(
    barmode = 'group',
    title = 'Lengte van de zin per sentiment'
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='grouped-bar')

# Clustering Analyse van elk woord per sentiment

In [ ]:
from sklearn.cluster import KMeans
import numpy as np

x = np.array(train['WordCount'])
km = KMeans(n_clusters = 4)
km.fit(x.reshape(-1,1))  
train['cluster'] = list(km.labels_)

In [ ]:
y = np.array(test['WordCount'])
km = KMeans(n_clusters = 4)
km.fit(y.reshape(-1,1))  
test['cluster'] = list(km.labels_)

In [ ]:
cluster = train.groupby(['Sentiment','cluster'])['WordCount'].describe().reset_index()
cluster

Weer is te zien dat sentiment 2 een hoog aantal heeft. Een groot deel van de woorden zijn dus van neutrale sentiment.

In [ ]:
train.groupby(['Sentiment','cluster'])['WordCount'].count().unstack().plot(kind='bar', stacked=False)
train.groupby(['Sentiment','cluster'])['WordCount'].mean().unstack().plot(kind='bar', stacked=False)
train.groupby(['Sentiment','cluster'])['WordCount'].min().unstack().plot(kind='bar', stacked=False)
train.groupby(['Sentiment','cluster'])['WordCount'].max().unstack().plot(kind='bar', stacked=False)

In [ ]:
gc.collect()

## Het voorbereiden van een model voor LSTM en Clustering
#### Het verdelen van de Train Set tot train set en validation set

In [ ]:
train_text = train.filter(['CleanedPhrase','cluster'])
test_text = test.filter(['CleanedPhrase','cluster'])
target = train.Sentiment.values
y = to_categorical(target)

print(train_text.shape,target.shape,y.shape)

In [ ]:
X_train_text, X_val_text, y_train, y_val = train_test_split(train_text, y, test_size = 0.2, stratify = y, random_state = 123) # split train + validation

print(X_train_text.shape, y_train.shape)
print(X_val_text.shape, y_val.shape)

#### Zoek het aantal unieke woorden in train set

In [ ]:
all_words = ' '.join(X_train_text.CleanedPhrase.values)
all_words = word_tokenize(all_words)
dist = FreqDist(all_words)
unique_word_count = len(dist)
unique_word_count

#### Zoek de max. lengte van een review in train set

In [ ]:
review_length = []
for text in X_train_text.CleanedPhrase.values:
    word = word_tokenize(text)
    l = len(word)
    review_length.append(l)
    
max_review_length = np.max(review_length)
max_review_length

In [ ]:
max_features = unique_word_count
max_words = max_review_length
batch_size = 128
epochs = 3
num_classes=5

#### Tokenize text

Tokenizer zet strings om in token objects. Deze tokens kunnen heel makkelijk worden opgeteld of andere handelingen toegepast.

In [ ]:
tokenizer = Tokenizer(num_words = max_features)
tokenizer.fit_on_texts(list(X_train_text.CleanedPhrase.values))
X_train = tokenizer.texts_to_sequences(X_train_text.CleanedPhrase.values)
X_val = tokenizer.texts_to_sequences(X_val_text.CleanedPhrase.values)
X_test = tokenizer.texts_to_sequences(test.CleanedPhrase.values)

#### Sequence padding

Sequence padding zodat we de maximum lengte kunnen weten per data set.

In [ ]:
X_train = sequence.pad_sequences(X_train, maxlen = max_words)
X_val = sequence.pad_sequences(X_val, maxlen = max_words)
X_test = sequence.pad_sequences(X_test, maxlen = max_words)
print(X_train.shape, X_val.shape, X_test.shape)

### Clustering in X_train, X_test en X_val

In [ ]:
X_train = numpy.insert(X_train, 48, numpy.array([X_train_text.cluster.values]), axis = 1)
X_val = numpy.insert(X_val, 48, numpy.array([X_val_text.cluster.values]), axis = 1)
X_test = numpy.insert(X_test, 48, numpy.array([test.cluster.values]), axis=1)
print(X_train.shape, X_val.shape, X_test.shape)

In [ ]:
gc.collect()

#### Maak een model ervan

In [ ]:
model = Sequential()
model.add(Embedding(max_features, 100, mask_zero = True))
model.add(LSTM(64, dropout = 0.4, recurrent_dropout = 0.4, return_sequences = True))
model.add(LSTM(32, dropout = 0.5, recurrent_dropout = 0.5, return_sequences = False))
model.add(Dense(num_classes, activation = 'softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = Adam(lr = 0.001), metrics = ['accuracy'])
model.summary()

### Het trainen van de A.I.
Nu kan de A.I. trainen en zich valideren!

In [ ]:
%%time
history = model.fit(X_train, y_train, validation_data = (X_val, y_val), epochs = epochs, batch_size = batch_size, verbose=1)

#### Genereer het Prediction bestand

In [ ]:
prediction = model.predict_classes(X_test, verbose = 1)


In [ ]:
sub.Sentiment = prediction
sub.to_csv('sub.csv', index = False)
sub.head()

In [ ]:
unique, counts = numpy.unique(prediction, return_counts = True)
dict(zip(unique, counts))

Het bestand is onderaan de notebook te bekijken.